In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import tensorflow
import os
import cv2
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, LeakyReLU, GlobalAveragePooling2D, Rescaling
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.applications import ResNet50
from skimage.transform import resize
from keras.applications.resnet50 import preprocess_input

# Create and compile NN Models

In [ ]:
def create_nn_model_1():
  model = Sequential(name='Model_1')
  model.add(Conv2D(64, 2, activation = 'relu', input_shape = (600, 300, 3)))
  model.add(MaxPooling2D(pool_size = (2,2)))
  model.add(Dropout(0.1))
  model.add(Conv2D(128, 2, activation = 'relu'))
  model.add(MaxPooling2D(pool_size = (2,2)))
  model.add(Dropout(0.1))
  model.add(Conv2D(256, 2, activation = 'relu'))
  model.add(MaxPooling2D(pool_size = (4,4)))
  model.add(Dropout(0.1))
  model.add(Conv2D(512, 2, activation = 'relu'))
  model.add(MaxPooling2D(pool_size = (8,8)))
  model.add(Dropout(0.1))

  model.add(Flatten())

  model.add(Dense(2048, activation = 'relu'))
  model.add(Dense(1024, activation = 'relu'))
  model.add(Dense(256, activation = 'relu'))
  model.add(Dense(128, activation = 'relu'))

  model.add(Dense(10, activation = 'softmax'))
  # model.summary()

  return model

In [ ]:
def create_nn_model_2():
  model = Sequential(name='Model_2')
  model.add(Conv2D(64, 2, activation = 'relu', input_shape = (600, 300, 3)))
  model.add(MaxPooling2D(pool_size = (2,2)))
  model.add(Dropout(0.5))
  model.add(Conv2D(32, 2, activation = 'relu'))
  model.add(MaxPooling2D(pool_size = (2,2)))
  model.add(Dropout(0.5))
  model.add(Conv2D(32, 2, activation = 'relu'))
  model.add(MaxPooling2D(pool_size = (2,2)))

  model.add(Flatten())

  model.add(Dense(64, activation = 'relu'))
  model.add(BatchNormalization())
  model.add(Dense(32, activation = 'relu'))
  model.add(BatchNormalization())
  model.add(Dense(16, activation = 'relu'))
  model.add(BatchNormalization())

  model.add(Dense(10, activation = 'softmax'))
  # model.summary()

  return model

In [ ]:
def model_1_compile(model, checkpoint_path):
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.01)
  checkpoint = ModelCheckpoint(filepath=checkpoint_path, monitor='val_accuracy', mode='max', save_best_only=True, verbose=1)

  return model, reduce_lr, checkpoint

In [ ]:
def create_nn_model_3():
  model = Sequential(name='Model_3')
  model.add(Conv2D(64, (3, 3), input_shape = (600, 300, 3), padding="same", activation='relu'))
  model.add(Conv2D(64, (3, 3), strides=(4, 4), padding="same", activation='relu'))
  model.add(Dropout(0.25))
  model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
  model.add(Conv2D(128, (3, 3), strides=(4, 4), padding='same', activation='relu'))
  model.add(Dropout(0.25))

  model.add(Flatten())

  model.add(Dense(1024, activation=LeakyReLU(alpha=0.1)))
  model.add(BatchNormalization())

  model.add(Dense(512, activation=LeakyReLU(alpha=0.1)))
  model.add(BatchNormalization())

  model.add(Dense(256, activation=LeakyReLU(alpha=0.1)))
  model.add(BatchNormalization())

  model.add(Dense(10, activation = 'softmax'))
  # model.summary()

  return model

In [ ]:
def create_nn_model_4():
  resnet = ResNet50(weights='imagenet', include_top=False, classes=10)
  print("ResNet50 layers:",len(resnet.layers))

  model = resnet.output
  model = GlobalAveragePooling2D()(model)
  model = Dense(1024, activation = 'relu')(model)
  model = Dense(512, activation = 'relu')(model)
  last_layer = Dense(10, activation = 'softmax')(model)
  model = Model(name="Model_ResNet50", inputs = resnet.input, outputs = last_layer)

  print("Combined model:",len(model.layers),'layers')

  for layer in model.layers[:175]:
    layer.trainable = False

  for layer in model.layers[175:]:
      layer.trainable = True

  return model

In [ ]:
def resnet_compile(model, checkpoint_path):
  lr_schedule = tensorflow.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0001,
    decay_steps=10000,
    decay_rate=1e-6)

  optimizer = RMSprop(learning_rate=lr_schedule)

  model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  checkpoint = ModelCheckpoint(filepath=checkpoint_path, monitor='val_accuracy', mode='max', save_best_only=True, verbose=1)

  return model, checkpoint

# Audio files length: original MEL

In [ ]:
!unzip '/content/drive/MyDrive/GoogleCollab/Data/GTZAN/spectrograms/Spectrograms_with_test_train_split/spectrograms_mel.zip' -d "/content"

Archive:  /content/drive/MyDrive/GoogleCollab/Data/GTZAN/spectrograms/Spectrograms_with_test_train_split/spectrograms_mel.zip
   creating: /content/test/
   creating: /content/test/blues/
  inflating: /content/test/blues/blues.00006.png  
  inflating: /content/test/blues/blues.00007.png  
  inflating: /content/test/blues/blues.00010.png  
  inflating: /content/test/blues/blues.00011.png  
  inflating: /content/test/blues/blues.00018.png  
  inflating: /content/test/blues/blues.00022.png  
  inflating: /content/test/blues/blues.00023.png  
  inflating: /content/test/blues/blues.00031.png  
  inflating: /content/test/blues/blues.00032.png  
  inflating: /content/test/blues/blues.00036.png  
  inflating: /content/test/blues/blues.00037.png  
  inflating: /content/test/blues/blues.00047.png  
  inflating: /content/test/blues/blues.00064.png  
  inflating: /content/test/blues/blues.00065.png  
  inflating: /content/test/blues/blues.00066.png  
  inflating: /content/test/blues/blues.00069.pn

## Split data

In [ ]:
spectrograms_test_dir = '/content/spectrograms_mel/test'
spectrograms_train_dir = '/content/spectrograms_mel/train'

train_ds = tensorflow.keras.utils.image_dataset_from_directory(
  spectrograms_train_dir,
  label_mode='categorical',
  seed=123,
  image_size=(600, 300))

test_ds = tensorflow.keras.utils.image_dataset_from_directory(
  spectrograms_test_dir,
  label_mode='categorical',
  seed=123,
  image_size=(600, 300))

Found 798 files belonging to 10 classes.
Found 200 files belonging to 10 classes.


In [ ]:
def preprocess(images, labels):
  return tensorflow.keras.applications.resnet50.preprocess_input(images), labels

train_ds = train_ds.map(preprocess)

In [ ]:
# model = create_nn_model_1()
# model = create_nn_model_4()
# model = create_nn_model_3()
model = create_nn_model_2()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
checkpoint_dir = "/content/drive/MyDrive/GoogleCollab/Data/GTZAN/checkpoints/"
checkpoint_subdir = os.path.join(checkpoint_dir, model.name)
if not os.path.exists(checkpoint_subdir):
  os.makedirs(checkpoint_subdir)

checkpoint_path = os.path.join(checkpoint_subdir, f"best_model_original_mel.keras")

In [ ]:
# model, checkpoint = resnet_compile(model, checkpoint_path)
model, reduce_lr, checkpoint = model_1_compile(model, checkpoint_path)

## Train models

In [ ]:
import shutil

def save_history(history):
  filename = f'{model.name}_history_original_mel.npy'
  np.save(filename, H.history)
  content_dir = "/content/" + filename
  checkpoint_dir = "/content/drive/MyDrive/GoogleCollab/Data/GTZAN/checkpoints/" + model.name
  shutil.copy(content_dir, checkpoint_dir)

### ResNet model

In [ ]:
H = model.fit(train_ds, epochs=75, validation_data=test_ds, callbacks=[checkpoint])
save_history(H)

Epoch 1/75
25/25 [==============================] - ETA: 0s - loss: 1.9113 - accuracy: 0.3484
Epoch 1: val_accuracy improved from -inf to 0.42500, saving model to /content/drive/MyDrive/GoogleCollab/Data/GTZAN/checkpoints/Model_ResNet50/best_model_original_mel.h5
25/25 [==============================] - 20s 642ms/step - loss: 1.9113 - accuracy: 0.3484 - val_loss: 1.5788 - val_accuracy: 0.4250
Epoch 2/75
25/25 [==============================] - ETA: 0s - loss: 1.4803 - accuracy: 0.4950
Epoch 2: val_accuracy improved from 0.42500 to 0.53500, saving model to /content/drive/MyDrive/GoogleCollab/Data/GTZAN/checkpoints/Model_ResNet50/best_model_original_mel.h5
25/25 [==============================] - 16s 605ms/step - loss: 1.4803 - accuracy: 0.4950 - val_loss: 1.3855 - val_accuracy: 0.5350
Epoch 3/75
25/25 [==============================] - ETA: 0s - loss: 1.2989 - accuracy: 0.5789
Epoch 3: val_accuracy improved from 0.53500 to 0.57500, saving model to /content/drive/MyDrive/GoogleCollab/Dat

### CNN Model_1

In [ ]:
H = model.fit(train_ds, epochs=50, validation_data=test_ds, callbacks=[checkpoint, reduce_lr])
save_history(H)

Epoch 1/50
25/25 [==============================] - ETA: 0s - loss: 37.8100 - accuracy: 0.1015
Epoch 1: val_accuracy improved from -inf to 0.15500, saving model to /content/drive/MyDrive/GoogleCollab/Data/GTZAN/checkpoints/Model_1/best_model_original_mel.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


25/25 [==============================] - 40s 1s/step - loss: 37.8100 - accuracy: 0.1015 - val_loss: 2.2590 - val_accuracy: 0.1550 - lr: 0.0010
Epoch 2/50
25/25 [==============================] - ETA: 0s - loss: 2.1766 - accuracy: 0.1792
Epoch 2: val_accuracy improved from 0.15500 to 0.22000, saving model to /content/drive/MyDrive/GoogleCollab/Data/GTZAN/checkpoints/Model_1/best_model_original_mel.h5
25/25 [==============================] - 21s 816ms/step - loss: 2.1766 - accuracy: 0.1792 - val_loss: 2.1355 - val_accuracy: 0.2200 - lr: 0.0010
Epoch 3/50
25/25 [==============================] - ETA: 0s - loss: 2.0346 - accuracy: 0.2419
Epoch 3: val_accuracy improved from 0.22000 to 0.26500, saving model to /content/drive/MyDrive/GoogleCollab/Data/GTZAN/checkpoints/Model_1/best_model_original_mel.h5
25/25 [==============================] - 23s 889ms/step - loss: 2.0346 - accuracy: 0.2419 - val_loss: 2.0643 - val_accuracy: 0.2650 - lr: 0.0010
Epoch 4/50
25/25 [=============================

### CNN Model_2

In [ ]:
H = model.fit(train_ds, epochs=200, validation_data=test_ds, callbacks=[checkpoint, reduce_lr])
save_history(H)

Epoch 1/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 700ms/step - accuracy: 0.1844 - loss: 2.5155
Epoch 1: val_accuracy improved from -inf to 0.13500, saving model to /content/drive/MyDrive/GoogleCollab/Data/GTZAN/checkpoints/Model_2/best_model_original_mel.keras
25/25 ━━━━━━━━━━━━━━━━━━━━ 40s 963ms/step - accuracy: 0.1864 - loss: 2.5094 - val_accuracy: 0.1350 - val_loss: 2.2613 - learning_rate: 0.0010
Epoch 2/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step - accuracy: 0.3538 - loss: 1.8908
Epoch 2: val_accuracy did not improve from 0.13500
25/25 ━━━━━━━━━━━━━━━━━━━━ 11s 246ms/step - accuracy: 0.3541 - loss: 1.8897 - val_accuracy: 0.1050 - val_loss: 3.5002 - learning_rate: 0.0010
Epoch 3/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - accuracy: 0.4008 - loss: 1.7263
Epoch 3: val_accuracy did not improve from 0.13500
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 209ms/step - accuracy: 0.4017 - loss: 1.7234 - val_accuracy: 0.1000 - val_loss: 6.9595 - learning_rate: 0.0010
Epoch 4/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 155m

In [ ]:
!cp /content/model_2_mel.h5 /content/drive/MyDrive/GoogleCollab/Data/GTZAN/checkpoints/

### CNN Model_3

In [ ]:
H = model.fit(train_ds, epochs=200, validation_data=test_ds, callbacks=[checkpoint, reduce_lr])
save_history(H)

Epoch 1/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 848ms/step - accuracy: 0.2684 - loss: 2.5486
Epoch 1: val_accuracy improved from -inf to 0.16500, saving model to /content/drive/MyDrive/GoogleCollab/Data/GTZAN/checkpoints/Model_3/best_model_original_mel.keras
25/25 ━━━━━━━━━━━━━━━━━━━━ 70s 2s/step - accuracy: 0.2702 - loss: 2.5366 - val_accuracy: 0.1650 - val_loss: 43.1497 - learning_rate: 0.0010
Epoch 2/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 278ms/step - accuracy: 0.4184 - loss: 1.6275
Epoch 2: val_accuracy did not improve from 0.16500
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 340ms/step - accuracy: 0.4196 - loss: 1.6247 - val_accuracy: 0.1250 - val_loss: 11.8138 - learning_rate: 0.0010
Epoch 3/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step - accuracy: 0.5094 - loss: 1.3361
Epoch 3: val_accuracy improved from 0.16500 to 0.18000, saving model to /content/drive/MyDrive/GoogleCollab/Data/GTZAN/checkpoints/Model_3/best_model_original_mel.keras
25/25 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - accuracy: 0.5105 - loss: 1.333

AttributeError: 'Sequential' object has no attribute '_name'

# Audio files length: original MFCC

In [ ]:
!unzip '/content/drive/MyDrive/GoogleCollab/Data/GTZAN/spectrograms/Spectrograms_with_test_train_split/spectrograms_mfcc.zip' -d "/content/spectrograms_mfcc"

Archive:  /content/drive/MyDrive/GoogleCollab/Data/GTZAN/spectrograms/Spectrograms_with_test_train_split/spectrograms_mfcc.zip
   creating: /content/spectrograms_mfcc/test/
   creating: /content/spectrograms_mfcc/test/blues/
  inflating: /content/spectrograms_mfcc/test/blues/blues.00006.png  
  inflating: /content/spectrograms_mfcc/test/blues/blues.00007.png  
  inflating: /content/spectrograms_mfcc/test/blues/blues.00010.png  
  inflating: /content/spectrograms_mfcc/test/blues/blues.00011.png  
  inflating: /content/spectrograms_mfcc/test/blues/blues.00018.png  
  inflating: /content/spectrograms_mfcc/test/blues/blues.00022.png  
  inflating: /content/spectrograms_mfcc/test/blues/blues.00023.png  
  inflating: /content/spectrograms_mfcc/test/blues/blues.00031.png  
  inflating: /content/spectrograms_mfcc/test/blues/blues.00032.png  
  inflating: /content/spectrograms_mfcc/test/blues/blues.00036.png  
  inflating: /content/spectrograms_mfcc/test/blues/blues.00037.png  
  inflating: /co

## Split data

In [ ]:
spectrograms_test_dir = '/content/spectrograms_mfcc/test'
spectrograms_train_dir = '/content/spectrograms_mfcc/train'

train_ds = tensorflow.keras.utils.image_dataset_from_directory(
  spectrograms_train_dir,
  label_mode='categorical',
  seed=123,
  image_size=(600, 300))

test_ds = tensorflow.keras.utils.image_dataset_from_directory(
  spectrograms_test_dir,
  label_mode='categorical',
  seed=123,
  image_size=(600, 300))

Found 798 files belonging to 10 classes.
Found 200 files belonging to 10 classes.


In [ ]:
def preprocess(images, labels):
  return tensorflow.keras.applications.resnet50.preprocess_input(images), labels

train_ds = train_ds.map(preprocess)

In [ ]:
# model = create_nn_model_1()
# model = create_nn_model_4()
# model = create_nn_model_3()
model = create_nn_model_2()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
checkpoint_dir = "/content/drive/MyDrive/GoogleCollab/Data/GTZAN/checkpoints/"
checkpoint_subdir = os.path.join(checkpoint_dir, model.name)
if not os.path.exists(checkpoint_subdir):
  os.makedirs(checkpoint_subdir)

checkpoint_path = os.path.join(checkpoint_subdir, f"best_model_original_mfcc.keras")

## Train models

In [ ]:
import shutil

def save_history(history):
  filename = f'{model.name}_history_original_mfcc.npy'
  np.save(filename, H.history)
  content_dir = "/content/" + filename
  checkpoint_dir = "/content/drive/MyDrive/GoogleCollab/Data/GTZAN/checkpoints/" + model.name
  shutil.copy(content_dir, checkpoint_dir)

### ResNet model

In [ ]:
model, checkpoint = resnet_compile(model, checkpoint_path)
H = model.fit(train_ds, epochs=100, validation_data=test_ds, callbacks=[checkpoint])
save_history(H)

Epoch 1/100
25/25 [==============================] - ETA: 0s - loss: 2.0985 - accuracy: 0.2469
Epoch 1: val_accuracy improved from -inf to 0.31000, saving model to /content/drive/MyDrive/GoogleCollab/Data/GTZAN/checkpoints/Model_ResNet50/best_model_original_mfcc.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


25/25 [==============================] - 34s 831ms/step - loss: 2.0985 - accuracy: 0.2469 - val_loss: 1.8515 - val_accuracy: 0.3100
Epoch 2/100
25/25 [==============================] - ETA: 0s - loss: 1.8092 - accuracy: 0.3647
Epoch 2: val_accuracy improved from 0.31000 to 0.39500, saving model to /content/drive/MyDrive/GoogleCollab/Data/GTZAN/checkpoints/Model_ResNet50/best_model_original_mfcc.h5
25/25 [==============================] - 14s 514ms/step - loss: 1.8092 - accuracy: 0.3647 - val_loss: 1.6912 - val_accuracy: 0.3950
Epoch 3/100
25/25 [==============================] - ETA: 0s - loss: 1.6895 - accuracy: 0.3985
Epoch 3: val_accuracy improved from 0.39500 to 0.40000, saving model to /content/drive/MyDrive/GoogleCollab/Data/GTZAN/checkpoints/Model_ResNet50/best_model_original_mfcc.h5
25/25 [==============================] - 14s 527ms/step - loss: 1.6895 - accuracy: 0.3985 - val_loss: 1.6106 - val_accuracy: 0.4000
Epoch 4/100
25/25 [==============================] - ETA: 0s - los

### CNN Model_1

In [ ]:
model, reduce_lr, checkpoint = model_1_compile(model, checkpoint_path)
H = model.fit(train_ds, epochs=100, validation_data=test_ds, callbacks=[checkpoint, reduce_lr])
save_history(H)

Epoch 1/100
25/25 [==============================] - ETA: 0s - loss: 36.0836 - accuracy: 0.1103
Epoch 1: val_accuracy improved from -inf to 0.10000, saving model to /content/drive/MyDrive/GoogleCollab/Data/GTZAN/checkpoints/Model_1/best_model_original_mfcc.h5
25/25 [==============================] - 24s 838ms/step - loss: 36.0836 - accuracy: 0.1103 - val_loss: 2.3034 - val_accuracy: 0.1000 - lr: 0.0010
Epoch 2/100
25/25 [==============================] - ETA: 0s - loss: 2.3201 - accuracy: 0.0977
Epoch 2: val_accuracy did not improve from 0.10000
25/25 [==============================] - 19s 713ms/step - loss: 2.3201 - accuracy: 0.0977 - val_loss: 2.3010 - val_accuracy: 0.0950 - lr: 0.0010
Epoch 3/100
25/25 [==============================] - ETA: 0s - loss: 2.2985 - accuracy: 0.1216
Epoch 3: val_accuracy improved from 0.10000 to 0.11000, saving model to /content/drive/MyDrive/GoogleCollab/Data/GTZAN/checkpoints/Model_1/best_model_original_mfcc.h5
25/25 [==============================] - 

### CNN Model_2

In [ ]:
model, reduce_lr, checkpoint = model_1_compile(model, checkpoint_path)
H = model.fit(train_ds, epochs=200, validation_data=test_ds, callbacks=[checkpoint, reduce_lr])
save_history(H)

Epoch 1/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 473ms/step - accuracy: 0.1358 - loss: 2.6134
Epoch 1: val_accuracy improved from -inf to 0.10000, saving model to /content/drive/MyDrive/GoogleCollab/Data/GTZAN/checkpoints/Model_2/best_model_original_mfcc.keras
25/25 ━━━━━━━━━━━━━━━━━━━━ 34s 848ms/step - accuracy: 0.1363 - loss: 2.6091 - val_accuracy: 0.1000 - val_loss: 2.4242 - learning_rate: 0.0010
Epoch 2/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step - accuracy: 0.2109 - loss: 2.2069
Epoch 2: val_accuracy did not improve from 0.10000
25/25 ━━━━━━━━━━━━━━━━━━━━ 15s 183ms/step - accuracy: 0.2112 - loss: 2.2056 - val_accuracy: 0.1000 - val_loss: 3.8011 - learning_rate: 0.0010
Epoch 3/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step - accuracy: 0.2718 - loss: 2.0900
Epoch 3: val_accuracy did not improve from 0.10000
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 308ms/step - accuracy: 0.2718 - loss: 2.0890 - val_accuracy: 0.0850 - val_loss: 6.1275 - learning_rate: 0.0010
Epoch 4/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 167

In [ ]:
model.save('/content/model_2_mfcc.h5', save_format='h5')
!cp /content/model_2_mfcc.h5 /content/drive/MyDrive/GoogleCollab/Data/GTZAN/checkpoints/

### CNN Model_3


In [ ]:
model, reduce_lr, checkpoint = model_1_compile(model, checkpoint_path)
H = model.fit(train_ds, epochs=200, validation_data=test_ds, callbacks=[checkpoint, reduce_lr])
save_history(H)

Epoch 1/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 397ms/step - accuracy: 0.1585 - loss: 2.8087
Epoch 1: val_accuracy improved from -inf to 0.10000, saving model to /content/drive/MyDrive/GoogleCollab/Data/GTZAN/checkpoints/Model_3/best_model_original_mfcc.keras
25/25 ━━━━━━━━━━━━━━━━━━━━ 60s 2s/step - accuracy: 0.1601 - loss: 2.7973 - val_accuracy: 0.1000 - val_loss: 49.1786 - learning_rate: 0.0010
Epoch 2/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step - accuracy: 0.3244 - loss: 1.8689
Epoch 2: val_accuracy improved from 0.10000 to 0.17000, saving model to /content/drive/MyDrive/GoogleCollab/Data/GTZAN/checkpoints/Model_3/best_model_original_mfcc.keras
25/25 ━━━━━━━━━━━━━━━━━━━━ 74s 2s/step - accuracy: 0.3249 - loss: 1.8677 - val_accuracy: 0.1700 - val_loss: 9.5001 - learning_rate: 0.0010
Epoch 3/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step - accuracy: 0.4034 - loss: 1.7163
Epoch 3: val_accuracy did not improve from 0.17000
25/25 ━━━━━━━━━━━━━━━━━━━━ 40s 258ms/step - accuracy: 0.4033 - loss: 1.7